In [19]:
#coding: utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RFC
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [2]:
train = pd.read_table("train.tsv")
test = pd.read_table("test.tsv")
train.head()

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,unacc,low,med,3,2,small,low
1,3,acc,low,high,3,more,small,med
2,7,unacc,vhigh,high,5more,2,small,med
3,11,acc,high,high,3,more,big,med
4,12,unacc,high,high,3,2,med,high


In [3]:
y = train["class"]
train = train.drop("class", axis=1)
train_dummies = pd.get_dummies(train)
train_dummies.head()

,id,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0
1,3,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1
2,7,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1
3,11,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1
4,12,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 7 columns):
id          864 non-null int64
buying      864 non-null object
maint       864 non-null object
doors       864 non-null object
persons     864 non-null object
lug_boot    864 non-null object
safety      864 non-null object
dtypes: int64(1), object(6)
memory usage: 47.3+ KB


In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_dummies, y, random_state=709)

In [21]:
params = {"n_estimators": [100, 150, 200, 300, 500], "max_depth": [10, 15, 20, 25, 30]}
score = "f1"
clf = GridSearchCV(RFC(), params, cv=5)

In [22]:
clf.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200, 300, 500], 'max_depth': [10, 15, 20, 25, 30]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [23]:
print(clf.best_params_)

{'max_depth': 15, 'n_estimators': 150}


In [26]:
forest = RFC(n_estimators=150, max_depth=15)
forest.fit(x_train, y_train)
forest.score(x_test, y_test)

0.9074074074074074

In [25]:
print("Best parameters set found on development set: %s" % clf.best_params_)
print()

# それぞれのパラメータでの試行結果の表示
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()

# テストデータセットでの分類精度を表示
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(x_test)
print(classification_report(y_true, y_pred))

Best parameters set found on development set: {'max_depth': 15, 'n_estimators': 150}

Grid scores on development set:

0.875 (+/-0.021) for {'max_depth': 10, 'n_estimators': 100}
0.889 (+/-0.033) for {'max_depth': 10, 'n_estimators': 150}
0.889 (+/-0.028) for {'max_depth': 10, 'n_estimators': 200}
0.887 (+/-0.019) for {'max_depth': 10, 'n_estimators': 300}
0.890 (+/-0.027) for {'max_depth': 10, 'n_estimators': 500}
0.884 (+/-0.019) for {'max_depth': 15, 'n_estimators': 100}
0.897 (+/-0.030) for {'max_depth': 15, 'n_estimators': 150}
0.887 (+/-0.020) for {'max_depth': 15, 'n_estimators': 200}
0.890 (+/-0.027) for {'max_depth': 15, 'n_estimators': 300}
0.897 (+/-0.020) for {'max_depth': 15, 'n_estimators': 500}
0.884 (+/-0.017) for {'max_depth': 20, 'n_estimators': 100}
0.884 (+/-0.022) for {'max_depth': 20, 'n_estimators': 150}
0.895 (+/-0.020) for {'max_depth': 20, 'n_estimators': 200}
0.887 (+/-0.020) for {'max_depth': 20, 'n_estimators': 300}
0.890 (+/-0.023) for {'max_depth': 20, 'n

In [30]:
test_dummies = pd.get_dummies(test)
sample = pd.read_csv("sample_submit.csv", header=None)
sample.head()
sample[1] = clf.predict(test_dummies)
sample.to_csv("submit2.csv", header=None, index=None)